# FlixStock's Prediction File

In [4]:
# Importing the necessary libraries 
import pandas as pd
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [5]:
 # Load the trained model
 loaded_model = tf.keras.models.load_model("./model/")

In [6]:
# Counting the number of test images
path = "./test/"
files = os.listdir(path)
print(len(files))

10


In [29]:
# Creating an image stack for making batch predictions
images = []
file_name = []
for file in files:
    file_name.append(file)
    image = cv2.imread(path+file)
    image = cv2.resize(image,dsize=(300,300))
    image = image.reshape((image.shape[0],image.shape[1],3))
    images.append(image)

In [17]:
# Converting the data from list to numpy array
features = np.zeros((10,300,300,3),dtype = 'float32')

for i in range(10):
    features[i] = images[i]

# Assuming that the image is an 8-bit image we normalize the pixels of the image
features = features/255

In [38]:
# Making predictions and getting the index of the max probability as that index is the predicted class
class_neck = []
class_sleeve = []
class_pattern = []
for i in range(10):
    test = features[i].reshape((1,300,300,3))
    val = loaded_model.predict(test)
    neck =  val[0]
    sleeve = val[1]
    pattern = val[2]
    class_neck.append(np.argmax(neck))
    class_sleeve.append(np.argmax(sleeve))
    class_pattern.append(np.argmax(pattern))

In [33]:
# Converting into final data frame
dict_pred = {'filename': file_name,'neck': class_neck,'sleeve_length': class_sleeve, 'pattern': class_pattern }
pred_df = pd.DataFrame.from_dict(dict_pred)
pred_df.head()
pred_df.to_csv("prediction.csv")